In [ ]:
import pandas as pd
import datetime
import os
import timeit
from ortools.linear_solver import pywraplp




load c:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\ortools.dll...


In [14]:
# Read Excel file
file_name = os.fsdecode("C:/Users/Administrator/VSCODE/BatteryOptimization/Data/Dataset_Optimization.xlsx")
workbook = pd.ExcelFile(file_name)

output_folder = "output"
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)

# Load timeseries data
marketDF = workbook.parse("Timeseries data")
marketDF = marketDF.iloc[:,:4]
marketDF.columns = ["time_slot", "HH1_PV_generation", "HH1_Load", "market_price_1"]
marketDF = marketDF[~pd.isnull(marketDF["time_slot"])].fillna(0)

market1DF = marketDF.copy()
market1DF.sort_values(by=["time_slot"], inplace=True)
market1DF["time_string"] = market1DF.apply(lambda x:(x["time_slot"]+ datetime.timedelta(seconds=0.002)).strftime("%d/%m/%Y %H:%M"), axis=1)
market1DF.set_index("time_string", inplace=True)
marketDF = market1DF

print (marketDF)
# Load battery data
battDF = workbook.parse("Battery")
battDF = battDF.iloc[:,:8]
battDF.columns = ["max_charge_rate", "max_discharge_rate", "capacity", "charge_eff", "discharge_eff", "min_soc", "max_soc", "initial_soc"]

# Convert dataframe to dictionary
marketDict = marketDF.to_dict()
battDict = battDF.to_dict()

timeInterval = marketDF.iloc[1]["time_slot"] - marketDF.iloc[0]["time_slot"]

# Assign the data to right places
input = type("input", (dict,), {})()
input.update({
    "simData": {
        "startTime": datetime.datetime.strptime(marketDF.index[0], "%d/%m/%Y %H:%M"),
        "dt": int(round(timeInterval.total_seconds())) / (60 * 60), #in hour
        "tIndex": marketDF.shape[0]
        },
    "market": {
        key: {
            sub_key: sub_item for sub_key, sub_item in marketDict[key].items()
            } for key in marketDict.keys() if key != "time_slot"
        },

    "batt": {
        key: item[0] for key, item in battDict.items()
        }
    })
print (input)

                           time_slot  HH1_PV_generation  HH1_Load  \
time_string                                                         
01/05/2017 00:00 2017-05-01 00:00:00              0.000     0.310   
01/05/2017 01:00 2017-05-01 01:00:00              0.000     0.280   
01/05/2017 02:00 2017-05-01 02:00:00              0.000     0.285   
01/05/2017 03:00 2017-05-01 03:00:00              0.000     0.166   
01/05/2017 04:00 2017-05-01 04:00:00              0.037     0.202   
...                              ...                ...       ...   
31/05/2017 19:00 2017-05-31 19:00:00              0.000     0.729   
31/05/2017 20:00 2017-05-31 20:00:00              0.000     1.370   
31/05/2017 21:00 2017-05-31 21:00:00              0.000     0.495   
31/05/2017 22:00 2017-05-31 22:00:00              0.000     0.330   
31/05/2017 23:00 2017-05-31 23:00:00              0.000     0.360   

                  market_price_1  
time_string                       
01/05/2017 00:00         0.30171

In [15]:
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver.CreateSolver("CBC")

inf = solver.infinity()

tIndex = input["simData"]["tIndex"] # number of timeslots
dt = input["simData"]["dt"] # time interval in hour

# Create datetime array
startTime = input["simData"]["startTime"].strftime("%d/%m/%Y %H:%M")
tIndex = input["simData"]["tIndex"]
timestamp = pd.date_range(startTime, periods=tIndex, freq=str(dt * 60) + "min")
time = [timestamp[i].strftime("%d/%m/%Y %H:%M") for i in range(len(timestamp))]

time_s = timeit.default_timer()
# Add timeseries variables
vGrid = [solver.NumVar(lb=-inf, ub=inf, name="") for _ in range(tIndex)]

vBattPower = [solver.NumVar(lb=-inf, ub=inf, name="") for _ in range(tIndex)]
vCharge = [solver.NumVar(lb=-inf, ub=0, name="") for _ in range(tIndex)]
vDischarge = [solver.NumVar(lb=0, ub=inf, name="") for _ in range(tIndex)]
vChargeStatus = [solver.BoolVar(name="") for _ in range(tIndex)]
vSOC = [solver.NumVar(lb=0, ub=1, name="") for _ in range(tIndex)]

In [16]:
# Add constraints
for i in range(tIndex):
    
    t = time[i]
    
    
    # Battery constraints
    solver.Add(vBattPower[i] == vCharge[i] + vDischarge[i]) # Eqn. 4
    solver.Add(vCharge[i] >= -input["batt"]["max_charge_rate"] * vChargeStatus[i]) # Eqn. 5(a)
    solver.Add(vDischarge[i] <= input["batt"]["max_discharge_rate"] * (1-vChargeStatus[i])) # Eqn. 5(b)
    
    if i == 0:
        solver.Add(vSOC[i] == input["batt"]["initial_soc"] - dt / input["batt"]["capacity"] * (vCharge[i] * (1-input["batt"]["charge_eff"]) + vDischarge[i] / (1-input["batt"]["discharge_eff"]))) # Eqn. 6
    else:
        solver.Add(vSOC[i] == vSOC[i-1] - dt / input["batt"]["capacity"] * (vCharge[i] * (1-input["batt"]["charge_eff"]) + vDischarge[i] / (1-input["batt"]["discharge_eff"]))) # Eqn. 6
        
    solver.Add(vSOC[i] >= input["batt"]["min_soc"]) # Eqn. 7
    solver.Add(vSOC[i] <= input["batt"]["max_soc"]) # Eqn. 7

    # Add objective
obj = 0
obj += sum([vGrid[i] * input["market"]["market_price_1"][time[i]] * dt for i in range(tIndex)])
solver.Minimize(obj)

KeyError: '05/01/2017 00:00'